In [2]:
!git clone https://github.com/doaa1adel/UERC-Ear-Recognition-Challenge

Cloning into 'UERC-Ear-Recognition-Challenge'...
remote: Enumerating objects: 15, done.
remote: Total 15 (delta 0), reused 0 (delta 0), pack-reused 15
Unpacking objects: 100% (15/15), done.


In [3]:
cd /content/UERC-Ear-Recognition-Challenge

/content/UERC-Ear-Recognition-Challenge


In [6]:
import os,shutil
import cv2,keras
import numpy as np
from imgaug import augmenters as iaa
import skimage.io as io
from sklearn.model_selection import train_test_split
from keras.applications import ResNet50
from keras.optimizers import Adam

In [22]:
def create_dataset(path,num_classes,data_splitno):
    data_split=[]
    subjects = os.listdir(path) # getting each subject folders (no. of classes=166)
    for subject in subjects[:num_classes]:     ## selcting only 150 classes
        os.makedirs(os.path.join('/content/gdrive/My Drive/Colab Notebooks/Data/train',subject))  ## creating new folder for train and test with 60/40 split from each subjects
        os.makedirs(os.path.join('/content/gdrive/My Drive/Colab Notebooks/Data/test',subject))
        subject_images = os.listdir(os.path.join(path,subject))
        for image in subject_images:
            if(image.endswith('.png')):
                data_split.append(image)
                train_no = int(data_splitno*len(data_split))
        for i in range(0,len(data_split)):
        
            if(i<=train_no-1):
                source_train=os.path.join(path,subject,data_split[i])
                destination_train=os.path.join('/content/gdrive/My Drive/Colab Notebooks/Data/train',subject)
                shutil.copy(source_train,destination_train)
        
            else:
                source_test=os.path.join(path,subject,data_split[i])
                destination_test=os.path.join('/content/gdrive/My Drive/Colab Notebooks/Data/test',subject)
                shutil.copy(source_test,destination_test)
                
        data_split=[] 

In [14]:
def data_augmentation(image,path):
    flip =iaa.Sequential([iaa.Fliplr(0.5)]).augment_images(image)
    io.imsave(path+'flip.png',flip)
    
    crop = iaa.Sequential([iaa.Crop(percent=(0,0.01))]).augment_images(image)
    io.imsave(path+'crop.png',crop)
    
    GB = iaa.Sequential([iaa.GaussianBlur(sigma=(0,3.0))]).augment_images(image)
    io.imsave(path+'GB.png',GB)
    
    GN = iaa.Sequential([iaa.AdditiveGaussianNoise(scale=(0.0,0.2))]).augment_images(image)
    io.imsave(path+'GN.png',GN)
    
    CN = iaa.Sequential([iaa.ContrastNormalization(0.5,per_channel=0.5)]).augment_images(image)
    io.imsave(path+'CN.png',CN)
    
    BR = iaa.Sequential([iaa.Multiply((0.8,1.2))]).augment_images(image)
    io.imsave(path+'BR.png',BR)
    
    scale= iaa.Sequential([iaa.Affine(scale={"x": (0.8, 1.2), "y": (0.8,1.2)})]).augment_images(image)
    io.imsave(path+'scale.png',scale)
    
    rotate = iaa.Sequential([iaa.Affine(rotate=(-45,45))]).augment_images(image)
    io.imsave(path+'rotate.png',rotate)
    

In [15]:
def get_augmented_data(path):
    subjects = os.listdir(path)
    for subject in subjects:
        subject_path=os.path.join(path,subject)
        subject_images = os.listdir(subject_path)
        for image in subject_images:
            if(image.endswith('.png')):
                image_path=os.path.join(path,subject,image)
                image = cv2.resize(cv2.imread(image_path),(100,100))
                im_path=os.path.join(os.path.splitext(image_path)[0])
                data_augmentation(image,im_path)

In [16]:
def data(path):
    images=[]
    label=[]
    subjects = os.listdir(path)
    for subject in subjects:
        subject_path=os.path.join(path,subject)
        subject_images = os.listdir(subject_path)
        for image in subject_images:
            if(image.endswith('.png')):
                image_path=os.path.join(path,subject,image)
                image = cv2.resize(cv2.imread(image_path),(224,224))
                images.append(image)
                label.append(subject)
    
    images=np.array(images)
    label=np.array(label)
    return images,label

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
root_path = 'gdrive/My Drive/Colab Notebooks/'  #change dir to your project folder

In [23]:
!pwd  #to check the current path

/content/UERC-Ear-Recognition-Challenge


In [ ]:
#path = 'Dataset/Train Dataset' #the original command
path= '/content/gdrive/My Drive/Colab Notebooks/Train Dataset'
#train = 'Train Dataset'
num_classes=150
data_splitno=0.6
create_dataset(path,num_classes,data_splitno)

# Data Augmentation
train_path = '/content/gdrive/My Drive/Colab Notebooks/Data/train'
test_path = '/content/gdrive/My Drive/Colab Notebooks/Data/test'
get_augmented_data(train_path)
get_augmented_data(test_path)

#Getting train and test data(80/20)
train_images,train_label= data(train_path)
test_images,test_label= data(test_path)
x_train,x_valid,y_train,y_valid=train_test_split(train_images,train_label,test_size=0.2,random_state=42)
x_test=test_images
y_test=test_label

# convert class vectors to binary class matrices
y_train=keras.utils.to_categorical(y_train,1000)
y_valid= keras.utils.to_categorical(y_valid,1000)
y_test= keras.utils.to_categorical(y_test,1000)
print('x_train=',x_train.shape)
print('y_train=',y_train.shape)
print('x_test=',x_test.shape)
print('y_test=',y_test.shape)

In [3]:
model = ResNet50(include_top=True, weights='imagenet', input_tensor=None, input_shape=x_train[0].shape, 
                 pooling=None, classes=1000)   #weights='imagenet' #weights=None
model.summary()

NameError: ignored

In [1]:
model.compile(loss='categorical_crossentropy',optimizer=Adam(),metrics=['accuracy'])
model.fit(x_train, y_train,batch_size=70,epochs=1,verbose=1,validation_data=(x_valid, y_valid))


NameError: ignored

In [27]:
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 125618.20784785546
Test accuracy: 0.05775076150894165


In [31]:
results = np.argmax(model.predict(x_test),axis = 1)
results = np.reshape(results,(len(results),1))

#accuracy 

print("Accuracy: ",(sum(results == y_test))/len(results) * 100)

Accuracy:  [0.16886187 0.16886187 0.16886187 0.16886187 0.16886187 0.16886187
 0.16886187 0.16886187 0.16886187 0.16886187 0.16886187 0.16886187
 0.16886187 0.16886187 0.16886187 0.16886187 0.16886187 0.16886187
 0.16886187 0.16886187 0.16886187 0.16886187 0.16886187 0.16886187
 0.16886187 0.16886187 0.16886187 0.16886187 0.16886187 0.16886187
 0.16886187 0.16886187 0.16886187 0.16886187 0.16886187 0.16886187
 0.16886187 0.16886187 0.16886187 0.16886187 0.16886187 0.16886187
 0.16886187 0.16886187 0.16886187 0.16886187 0.16886187 0.16886187
 0.16886187 0.16886187 0.16886187 0.16886187 0.16886187 0.16886187
 0.16886187 0.16886187 0.16886187 0.16886187 0.16886187 0.16886187
 0.16886187 0.16886187 0.16886187 0.16886187 0.16886187 0.16886187
 0.16886187 0.16886187 0.16886187 0.16886187 0.16886187 0.16886187
 0.16886187 0.16886187 0.16886187 0.16886187 0.16886187 0.16886187
 0.16886187 0.16886187 0.16886187 0.16886187 0.16886187 0.16886187
 0.16886187 0.16886187 0.16886187 0.16886187 0.1688